In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import re
import nltk

In [2]:
#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

In [3]:
df = pd.read_csv("/kaggle/input/midyeardata/midYearFellowship.csv", index_col=0)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115 entries, 0 to 114
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Type of Program             114 non-null    object 
 1   Primary Setting             112 non-null    object 
 2   Description of opportunity  114 non-null    object 
 3   Preceptor                   114 non-null    object 
 4   Institution                 114 non-null    object 
 5   Length (Yrs)                114 non-null    float64
 6   Start Date                  114 non-null    object 
 7   Stipend                     98 non-null     object 
 8   Number of These Positions   114 non-null    float64
 9   Region                      104 non-null    object 
 10  Program Website             91 non-null     object 
 11  Contact                     114 non-null    object 
 12  Web ID                      114 non-null    object 
 13  Last Modified               114 non

In [5]:
# Find missing values in data
df.isna().sum()

Type of Program                1
Primary Setting                3
Description of opportunity     1
Preceptor                      1
Institution                    1
Length (Yrs)                   1
Start Date                     1
Stipend                       17
Number of These Positions      1
Region                        11
Program Website               24
Contact                        1
Web ID                         1
Last Modified                  1
dtype: int64

# Fill empty values with average stipend

In [6]:
# Clean empty stipend values
df['Stipend'] = df['Stipend'].fillna("$0.00")

In [7]:
# Create Series by assigning names
Region = pd.Series(df["Region"])
Institution = pd.Series(df["Institution"])
Stipend = pd.Series(df["Stipend"])
Description_of_opportunity = pd.Series(df["Description of opportunity"])

# using pandas series merge()
df2 = pd.merge(Institution, Region, right_index = True,
               left_index = True)

# using pandas series merge()
df2 = pd.merge(df2, Stipend, right_index = True,
               left_index = True)    
                          
# using pandas series merge()
df2 = pd.merge(df2, Description_of_opportunity, right_index = True,
               left_index = True)    
               
df2

,Institution,Region,Stipend,Description of opportunity
0,University of Southern California,Pacific,"$63,750.00",The one-year Academic Pharmacy Fellowship is d...
1,"Roosevelt University College of Science, Healt...",Midwest,"$50,000.00","Roosevelt University College of Science, Healt..."
2,Sullivan University College of Pharmacy and He...,Southeast,"$48,000.00",This 12-month academic fellowship at Sullivan ...
3,Xavier University of Louisiana,Southeast,$0.00,The Fellowship in Clinical Pharmacy Education ...
4,University of North Carolina,Southeast,$0.00,The Center for Innovative Pharmacy Education a...
...,...,...,...,...
110,University of Maryland,Southeast,$0.00,The University of Maryland Medical Center and ...
111,University of Illinois at Chicago,Midwest,"$52,000.00",The University of Illinois at Chicago (UIC) Co...
112,University of Michigan,Midwest,"$56,000.00",The faculty in the department are seeking appl...
113,Geisinger,Northeast,$0.00,Responsible for conducting research under the ...


In [8]:
stipend = []

In [9]:
for x in df2["Stipend"]: 
    x = x[1:]
    y = re.sub('[,]', "", x[0:-3])
    stipend.append(int(y))
   
print(stipend)

[63750, 50000, 48000, 0, 0, 54000, 50000, 50000, 55000, 0, 45000, 0, 49500, 0, 0, 130000, 0, 50000, 60000, 58000, 50000, 0, 50000, 55224, 0, 0, 55000, 54840, 0, 52000, 54800, 60000, 58000, 0, 53000, 50000, 50000, 42519, 50000, 55000, 53000, 53000, 0, 50000, 0, 50000, 0, 55000, 51000, 52000, 52000, 51000, 50000, 0, 50000, 50000, 52500, 50353, 52000, 55000, 51300, 52704, 72000, 52850, 0, 47500, 45221, 50000, 47460, 54144, 54000, 60000, 50000, 0, 0, 43000, 0, 60000, 60000, 60000, 47000, 0, 0, 40716, 52000, 50000, 0, 55000, 54144, 50000, 55000, 50000, 63750, 50000, 50000, 51000, 54000, 54000, 42500, 40000, 56000, 0, 0, 50000, 42000, 54000, 45000, 55000, 48500, 48000, 0, 52000, 56000, 0, 0]


In [10]:
# Get average stipend
total = 0
average = 0
for x in stipend:    
    
    if x == 0:
       pass
    else:
        total+=x
        average +=1
    
print(f"Total Stipend Value: {total}")
print(f"Average Stipend: {total/average}")

Total Stipend Value: 4606275
Average Stipend: 52945.68965517241


In [11]:
df2.drop(columns="Stipend", inplace=True)
df2["Stipend"] = stipend

In [12]:
#Replace missing values with average stipend
for x in range(0,len(df2["Stipend"])):
    if df2["Stipend"].loc[x] == 0:
        df2["Stipend"].loc[x] = 52945
    else:
        pass

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [13]:
df2.isna().sum()

Institution                    1
Region                        11
Description of opportunity     1
Stipend                        0
dtype: int64

# # Find amount of opportunities available per region  

In [14]:
Region_list = ['Pacific', 'Midwest', 'Southeast', 'Rocky Mountain', 'Northeast']

In [15]:
# Replace unknown region with unknown
for x in range(0,len(df2["Region"])):
    if df2["Region"].loc[x] not in Region_list:
        df2["Region"].loc[x] = 'Unknown'
    else:
        pass

In [16]:
df2.dropna(inplace=True)

In [17]:
df2.isna().sum()

Institution                   0
Region                        0
Description of opportunity    0
Stipend                       0
dtype: int64

In [18]:
# Amount of opportunities available per region 
df2.groupby(by='Region').agg('count')

,Institution,Description of opportunity,Stipend
Region,,,
Midwest,27,27,27
Northeast,25,25,25
Pacific,9,9,9
Rocky Mountain,5,5,5
Southeast,33,33,33
Unknown,15,15,15


# Find most common words in description

In [19]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import re
#stops = set(stopwords.words('english'))
#print(stops)

In [20]:
tokens_list = []

In [21]:
for x in range(0,len(df2["Description of opportunity"])):
    t = df2["Description of opportunity"].loc[x] 
    tokens_list.append(t)
    #tokens = nltk.word_tokenize(df2["Description of opportunity"][0])

In [22]:
token_words = ' '.join(tokens_list)
tokens = nltk.word_tokenize(token_words)
#token_l = nltk.Text(tokens)

In [23]:
stopWords = set(stopwords.words('english'))
stopWords_custom = ["The", ".", ",", ":",";", ")", "(","’","&","•","%","This"]
#words = word_tokenize(data)
wordsFiltered = []

for w in tokens:
    if w not in stopWords:
        if w in stopWords_custom:
            pass
        else:
            wordsFiltered.append(w)

In [24]:
from collections import Counter
most_occur = []
  
# Pass the split_it list to instance of Counter class.
Counter = Counter(wordsFiltered)
  
# most_common() produces k frequently encountered
# input values and their respective counts.
most_occur = Counter.most_common(100) 
#print(most_occur)

In [25]:
# 20 most common words in description
print(most_occur[:20])

[('research', 206), ('clinical', 134), ('program', 115), ('fellowship', 106), ('fellow', 102), ('Pharmacy', 84), ('experience', 81), ('training', 74), ('pharmacy', 74), ('health', 70), ('practice', 69), ('University', 56), ('skills', 54), ('teaching', 52), ('Fellowship', 47), ('care', 47), ('provide', 45), ('Health', 44), ('development', 40), ('drug', 40)]


# Get contact emails

In [26]:
# Create Series by assigning names
Preceptor = pd.Series(df["Preceptor"])
Institution = pd.Series(df["Institution"])
Contact = pd.Series(df["Contact"])

# using pandas series merge()
df3 = pd.merge(Preceptor, Institution, right_index = True,
               left_index = True)

# using pandas series merge()
df3 = pd.merge(df3, Contact, right_index = True,
               left_index = True)     
               
df3

,Preceptor,Institution,Contact
0,Dr. William Gong,University of Southern California,Dr. William GongUniversity of Southern Califor...
1,"Dan Majerczyk, Pharm.D. or Nicole Alvey, Pharm.D.","Roosevelt University College of Science, Healt...","Dan Majerczyk, Pharm.D. or Nicole Alvey, Pharm..."
2,Kim Elder,Sullivan University College of Pharmacy and He...,Kim ElderSullivan University College of Pharma...
3,Dana Jamero,Xavier University of Louisiana,Dana JameroXavier University of Louisiana1 Dre...
4,Susan Charamut,University of North Carolina,Susan CharamutUniversity of North Carolina301 ...
...,...,...,...
110,"Bruce Anderson, PharmD, DABAT, FACCT",University of Maryland,"Bruce Anderson, PharmD, DABAT, FACCTUniversity..."
111,"Jennie Jarrett, PharmD, BCPS, MMedEd, FCCP",University of Illinois at Chicago,"Jennie Jarrett, PharmD, BCPS, MMedEd, FCCPUniv..."
112,Karen Farris,University of Michigan,Karen FarrisUniversity of Michigan428 Church S...
113,Laney Jones,Geisinger,"Laney JonesGeisinger100 N Academy AveDanville,..."


In [27]:
df3.dropna(inplace=True)
#df3.isna().sum()

In [28]:
df["Contact"][0]

'Dr. William GongUniversity of Southern California1985 Zonal AvenueLos Angeles, CA 90089(323) 442-2625 (Phone)wgong@usc.edu'

In [29]:
# Filter out email from contact string
Email = []
for x in df3["Contact"]: 
    #x.split("(Phone)")
    #print(x)
    # Remove all characters before the character '-' from string
    listOfWords = x.split("(Phone)", 1)
    if len(listOfWords) > 0: 
        strValue = listOfWords[1]
    Email.append(strValue)

In [30]:
df3.drop(columns="Contact", inplace=True)
df3["Email"] = Email

In [31]:
# Export emails to csv
#df3.to_csv("midYearFellowshipEmails.csv",index=False)

# Create CSV file to use in Tableau

In [32]:
df4 = df2.drop(columns="Description of opportunity")

In [33]:
df4.isna().sum()

Institution    0
Region         0
Stipend        0
dtype: int64

In [34]:
df4.columns

Index(['Institution', 'Region', 'Stipend'], dtype='object')

In [35]:
# Export to csv file
#df4.to_csv("midYearFellowshipLight.csv", index=False)